## 1️⃣ Setup Environment

In [ ]:
# Install Unsloth and dependencies
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps trl peft accelerate bitsandbytes jsonlines python-dotenv

In [ ]:
# Verify GPU availability
import torch
print(f"🔥 PyTorch version: {torch.__version__}")
print(f"🎮 CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎯 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("⚠️ No GPU found! Please enable GPU runtime.")

## 2️⃣ Configuration

In [ ]:
import os
from dataclasses import dataclass
from typing import Optional

@dataclass
class TrainingConfig:
    """Configuration for model fine-tuning"""
    
    # Model
    base_model: str = "unsloth/llama-3-8b-bnb-4bit"
    max_seq_length: int = 2048
    load_in_4bit: bool = True
    
    # LoRA Configuration
    lora_r: int = 16
    lora_alpha: int = 16
    lora_dropout: float = 0.0
    target_modules: list = None
    
    # Training Hyperparameters
    batch_size: int = 2
    gradient_accumulation_steps: int = 4
    warmup_steps: int = 5
    max_steps: int = 60
    learning_rate: float = 2e-4
    fp16: bool = False
    bf16: bool = True
    
    # Optimizer
    optim: str = "adamw_8bit"
    weight_decay: float = 0.01
    lr_scheduler_type: str = "linear"
    
    # Logging
    logging_steps: int = 1
    
    # Output
    output_dir: str = "./outputs"
    logging_dir: str = "./logs"
    save_steps: int = 25
    
    # Data
    dataset_text_field: str = "text"
    packing: bool = False
    
    def __post_init__(self):
        if self.target_modules is None:
            self.target_modules = [
                "q_proj", "k_proj", "v_proj", "o_proj",
                "gate_proj", "up_proj", "down_proj"
            ]

# Initialize configuration
config = TrainingConfig()
print("✅ Configuration loaded")

## 3️⃣ Data Formatter

In [ ]:
import jsonlines
from typing import List, Dict, Optional
from pathlib import Path

class DataFormatter:
    """Handles conversion of various data formats to instruction format"""
    
    ALPACA_PROMPT = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:
{output}"""

    ALPACA_PROMPT_NO_INPUT = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:
{output}"""

    @staticmethod
    def format_alpaca(
        instruction: str,
        output: str,
        input_text: Optional[str] = None
    ) -> Dict[str, str]:
        if input_text and input_text.strip():
            text = DataFormatter.ALPACA_PROMPT.format(
                instruction=instruction,
                input=input_text,
                output=output
            )
        else:
            text = DataFormatter.ALPACA_PROMPT_NO_INPUT.format(
                instruction=instruction,
                output=output
            )
        return {"text": text}
    
    @staticmethod
    def load_and_format_jsonl(
        input_path: str,
        format_type: str = "alpaca"
    ) -> List[Dict[str, str]]:
        formatted_data = []
        with jsonlines.open(input_path) as reader:
            for obj in reader:
                instruction = obj.get("instruction", "")
                output = obj.get("output", "")
                input_text = obj.get("input", "")
                
                if not instruction or not output:
                    continue
                
                formatted = DataFormatter.format_alpaca(
                    instruction, output, input_text
                )
                formatted_data.append(formatted)
        return formatted_data

print("✅ DataFormatter loaded")

## 4️⃣ Create Sample Dataset (or Upload Your Own)

In [ ]:
# Create example dataset
import json

example_data = [
    {"instruction": "What is the capital of France?", "output": "The capital of France is Paris. It is located in the north-central part of the country and is known for its art, culture, and historical landmarks like the Eiffel Tower.", "input": ""},
    {"instruction": "Explain what machine learning is", "output": "Machine learning is a subset of artificial intelligence that enables computers to learn and improve from experience without being explicitly programmed. It uses algorithms and statistical models to analyze and draw inferences from patterns in data.", "input": ""},
    {"instruction": "Write a Python function to calculate factorial", "output": "Here's a Python function to calculate factorial:\n\ndef factorial(n):\n    if n == 0 or n == 1:\n        return 1\n    else:\n        return n * factorial(n - 1)\n\n# Example usage:\nprint(factorial(5))  # Output: 120", "input": ""},
    {"instruction": "Translate the following text to Spanish", "output": "¡Hola! ¿Cómo estás? Espero que estés teniendo un buen día.", "input": "Hello! How are you? I hope you're having a good day."},
    {"instruction": "What are the benefits of regular exercise?", "output": "Regular exercise offers numerous benefits including: improved cardiovascular health, stronger muscles and bones, better mental health and mood, enhanced immune system, weight management, increased energy levels, better sleep quality, and reduced risk of chronic diseases like diabetes and heart disease.", "input": ""}
]

# Save to file
with jsonlines.open('example_data.jsonl', mode='w') as writer:
    writer.write_all(example_data)

print(f"✅ Created example dataset with {len(example_data)} examples")
print("\n💡 To use your own data:")
print("   1. Upload your JSONL file using the file browser (left sidebar)")
print("   2. Change DATA_PATH below to your filename")

In [ ]:
# Set your data path here
DATA_PATH = "example_data.jsonl"  # Change this to your uploaded file

# Optional: Upload your own data
# from google.colab import files
# uploaded = files.upload()
# DATA_PATH = list(uploaded.keys())[0]

## 5️⃣ Load Model & Setup LoRA

In [ ]:
from unsloth import FastLanguageModel

print(f"📦 Loading model: {config.base_model}")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=config.base_model,
    max_seq_length=config.max_seq_length,
    dtype=None,  # Auto-detect
    load_in_4bit=config.load_in_4bit,
)

print("✅ Model loaded successfully")

# Setup LoRA
print("🔧 Setting up LoRA adapters...")

model = FastLanguageModel.get_peft_model(
    model,
    r=config.lora_r,
    target_modules=config.target_modules,
    lora_alpha=config.lora_alpha,
    lora_dropout=config.lora_dropout,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"✅ LoRA configured: {trainable_params:,} / {total_params:,} parameters trainable "
      f"({100 * trainable_params / total_params:.2f}%)")

## 6️⃣ Prepare Dataset

In [ ]:
from datasets import load_dataset

print(f"📁 Loading dataset from: {DATA_PATH}")

# Load dataset
dataset = load_dataset("json", data_files=DATA_PATH, split="train")

# Format if needed
if "text" not in dataset.column_names:
    print("⚠️ Formatting data...")
    formatted_data = DataFormatter.load_and_format_jsonl(DATA_PATH)
    
    # Save and reload
    with jsonlines.open('formatted_temp.jsonl', mode='w') as writer:
        writer.write_all(formatted_data)
    dataset = load_dataset("json", data_files='formatted_temp.jsonl', split="train")

print(f"✅ Dataset loaded: {len(dataset)} examples")
print(f"\n📋 Preview first example:")
print(dataset[0]['text'][:500] + "...")

## 7️⃣ Train the Model 🚀

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer
import os

print("🚀 Starting training...")
print(f"   Max steps: {config.max_steps}")
print(f"   Batch size: {config.batch_size}")
print(f"   Learning rate: {config.learning_rate}")

# Create output directory
os.makedirs(config.output_dir, exist_ok=True)

# Training arguments
training_args = TrainingArguments(
    output_dir=config.output_dir,
    per_device_train_batch_size=config.batch_size,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    warmup_steps=config.warmup_steps,
    max_steps=config.max_steps,
    learning_rate=config.learning_rate,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=config.logging_steps,
    optim=config.optim,
    weight_decay=config.weight_decay,
    lr_scheduler_type=config.lr_scheduler_type,
    seed=3407,
    save_steps=config.save_steps,
    save_total_limit=2,
    report_to="none",
)

# Initialize trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field=config.dataset_text_field,
    max_seq_length=config.max_seq_length,
    dataset_num_proc=2,
    packing=config.packing,
    args=training_args,
)

# Start training
trainer.train()

print("\n✅ Training completed!")

## 8️⃣ Save the Model

In [ ]:
# Save LoRA adapter
adapter_dir = "./outputs/adapter"
os.makedirs(adapter_dir, exist_ok=True)

print(f"💾 Saving adapter to: {adapter_dir}")
model.save_pretrained(adapter_dir)
tokenizer.save_pretrained(adapter_dir)

print("✅ Adapter saved successfully!")
print(f"   Files: adapter_config.json, adapter_model.safetensors")

# List saved files
print("\n📁 Saved files:")
!ls -lh {adapter_dir}

## 9️⃣ Test the Model (Optional)

In [ ]:
# Enable inference mode
FastLanguageModel.for_inference(model)

# Test prompt
test_instruction = "What is artificial intelligence?"
alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

inputs = tokenizer(
    [alpaca_prompt.format(test_instruction, "")],
    return_tensors="pt"
).to("cuda")

print("🤖 Generating response...\n")
outputs = model.generate(
    **inputs,
    max_new_tokens=128,
    temperature=0.7,
    top_p=0.9,
    do_sample=True
)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

## 🔟 Download Your Model

In [ ]:
# Zip the adapter folder for download
!zip -r adapter.zip ./outputs/adapter/

print("✅ Model zipped!")
print("\n📥 To download:")
print("   1. Find 'adapter.zip' in the file browser")
print("   2. Right-click → Download")
print("\nOr use the code below:")

from google.colab import files
files.download('adapter.zip')

---

## 🎉 Done!

You now have a fine-tuned LoRA adapter! 

### Next Steps:
1. Download your adapter
2. Load it locally with:
```python
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="path/to/adapter",
    max_seq_length=2048,
)
```

### Tips:
- **Longer training**: Increase `config.max_steps` to 200-500
- **Better results**: Use more training data (1000+ examples)
- **Different model**: Change `config.base_model` to other Unsloth models
- **Save merged model**: Uncomment section 8 alternative

### Resources:
- [Unsloth Documentation](https://github.com/unslothai/unsloth)
- [HuggingFace Models](https://huggingface.co/models)
- [Project GitHub](your-repo-url)

---

**Made with ❤️ for the AI community**